# Machine Learning for Cancer Prediction 
The data used for this tutorial is an RNA-seq gene expression data for different cancer types. The rows represent cancer samples and the columns represent gene count values. The last column contains the cancer categories.

The original data can found here: https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq#

## Required Libraries
 - numpy
 - matplotlib
 - seaborn
 - pandas
 - scikit

## Import Python libraries

In [ ]:
#data handling
import pandas as pd
import numpy as np

#data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import MinMaxScaler

#feature selection
from sklearn.feature_selection import mutual_info_classif

#classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

# performance metrics
from sklearn.metrics import balanced_accuracy_score,f1_score,precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score

## Read data

In [ ]:

#read data directly from a github repository

file_url='https://github.com/vappiah/Machine-Learning-Tutorials/raw/main/data/cancer_gene_expression.zip'

dataframe=pd.read_csv(file_url)



## Data Exploration & Cleaning




In [ ]:
#let's check the number of samples and features
#note:the last column contain the labels. it is not considered as a feature

print(dataframe.shape)

In [ ]:
#let's check some of the columns (first, second and third columns)
print(dataframe.columns[0:3])

In [ ]:
#lets check the name of the last column of this dataframe

dataframe.columns[-1]

In [ ]:
#check for missing values
datanul=dataframe.isnull().sum()
g=[i for i in datanul if i>0]

print('columns with missing values:%d'%len(g))

**GOOD JOB!!!!.** 

In [ ]:
#let's check how many different cancer types are there in the data
#note: in this tutorial the cancer types will be referred to as classes or labels

print(dataframe['Cancer_Type'].value_counts())

We can see that there are 5 classes/cancer types. And you can also see the number of samples diagnosed with a cancer type
      

In [ ]:
#plot a bar chat to display the class distribution

dataframe['Cancer_Type'].value_counts().plot.bar()

 
## **Data preprocesing** 
This is done to put the data in an appropriate format before modelling


In [ ]:
#we will now seperate the feature values from the class. we do this because scikit-learn requires that features and class are separated before parsing them to the classifiers.

X=dataframe.iloc[:,0:-1]
y=dataframe.iloc[:,-1]

\
**Encode labels**

This refers to converting the labels into a numeric form so as to convert them into the machine-readable form. 

In [ ]:
#let's encode target labels (y) with values between 0 and n_classes-1. We will then binarize the encoded labels.
#encoding is done using LabelEncoder and label_binarize is used to binarize the output.

label_encoder=LabelEncoder()
label_encoder.fit(y)
y_encoded=label_encoder.transform(y)
labels=label_encoder.classes_
classes=np.unique(y_encoded)
y = label_binarize(y_encoded, classes=classes)

In [ ]:
labels

In [ ]:
n_classes=y.shape[1]

In [ ]:
classes

\
**Data Splitting**\
We will now split the data into training and test subsets.
The training data is initially parsed to the machine learning model. this is to enable the model to identify discriminatory patterns which can be used to make future predictions.
The testing data is used to evaluate the model after the training phase.

In [ ]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

\
**Data Normalization**\
Data normalization is done so that the values are in the same range. This will improve model performance and avoid bias

In [ ]:
# scale data between 0 and 1

min_max_scaler=MinMaxScaler()
X_train_norm=min_max_scaler.fit_transform(X_train)
X_test_norm=min_max_scaler.fit_transform(X_test)

## **Feature Selection & Classification**
The purpose of feature selection is to select relevant features for classification. 
Feature selection is usually used as a pre-processing step before doing the actual learning. 

In this tutorial, mutual information algorithm is used to compute the relevance of each feature. The top n (eg. 300) features are selected for the machine learning analysis.

Classification is done using a Random Forest Classifier.

### Feature Selection using Mutual Information

In [ ]:
MI=mutual_info_classif(X_train_norm,np.argmax(y_train,axis=1))

In [ ]:
#select top n features. lets say 300.
#you can modify the value and see how the performance of the model changes

n_features=300
selected_scores_indices=np.argsort(MI)[::1][0:n_features]

In [ ]:
X_train_selected=X_train_norm[:,selected_scores_indices]
X_test_selected=X_test_norm[:,selected_scores_indices]

### Training
Training allows the machine learning model to learn from the data and use the identified patterns to predict the outcomes of data it has never seen before.
In the training phase, the model is given the training subset. In this tutorial, the Random Forest Classifier is used.

In [ ]:
#Random Forest Classifier
#because we are dealing with multiclass data, the one versus rest strategy is used.

RF=OneVsRestClassifier(RandomForestClassifier())
RF.fit(X_train_selected,y_train)
y_pred =RF.predict(X_test_selected)
pred_prob = RF.predict_proba(X_test_selected)

## Evaluating the model

After training the model, it is important to know how the model performs on an independent data(i.e. the test data). The following metrics are used to evaluate the model
- accuracy
- precision
- recall
- f1 score
- confusion matrix
- ROC Curve



In [ ]:
#accuracy
accuracy=np.round(balanced_accuracy_score(y_test.argmax(axis=1),y_pred.argmax(axis=1)),4)
print('accuracy:%0.4f'%accuracy)

#precision
precision=np.round(precision_score(y_test.argmax(axis=1),y_pred.argmax(axis=1),average = 'weighted'),4)
print('precision:%0.4f'%precision)

#recall
recall=np.round(recall_score(y_test.argmax(axis=1),y_pred.argmax(axis=1),average = 'weighted'),4)
print('recall:%0.4f'%recall)

#f1score
f1score=np.round(f1_score(y_test.argmax(axis=1),y_pred.argmax(axis=1),average = 'weighted'),4)
print('f1score:%0.4f'%f1score)


report=classification_report(y_test.argmax(axis=1),y_pred.argmax(axis=1), target_names=labels)
print('\n')
print('classification report\n\n')
print(report)

In [ ]:
#generate confusion matrix
cm=confusion_matrix(y_test.argmax(axis=1),y_pred.argmax(axis=1))
cm_df=pd.DataFrame(cm,index=labels,columns=labels)

In [ ]:
cm_df

In [ ]:
#plot the confusion matrix
sns.heatmap(cm_df,annot=True,cmap='Blues')
plt.xlabel('True Labels')
plt.ylabel('Predicted Labels')

In [ ]:
# roc curve for classes
fpr = {}
tpr = {}
thresh ={}
roc_auc = dict()

n_class = classes.shape[0]

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test[:,i], pred_prob[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
    # plotting    
    plt.plot(fpr[i], tpr[i], linestyle='--', 
             label='Class %d vs Rest (AUC=%0.2f)'%(i,roc_auc[i]))

plt.plot([0,1],[0,1],'b--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()    